In [ ]:
from datascience import *
import numpy as np
import matplotlib

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import warnings
warnings.simplefilter("ignore")

## Review

In [ ]:
def standard_units(x):
    return (x - np.mean(x))/np.std(x)

In [ ]:
ckd = Table.read_table('ckd.csv').relabeled('Blood Glucose Random', 'Glucose')
ckd.sample(3).show(3)

In [ ]:
ckd = ckd.select('Hemoglobin', 'Glucose', 'Class')

In [ ]:
ckd.sample(3).show(3)

______

In [ ]:
banknotes = Table.read_table('banknote.csv')
banknotes

## New material

Here's our example point for today.

In [ ]:
alice = make_array(0,1.1)
alice

### How can we find the *k*-nearest neighbors?

___

#### Detour: Rows and row objects

In [ ]:
type(ckd.row(0))

In [ ]:
ckd.row(0)

In [ ]:
ckd.row(0).item(1)

In [ ]:
make_array(ckd.row(0))

_____

First row in the training set:

In [ ]:
patient = ckd.drop('Class').row(0)

In [ ]:
make_array(patient)

#### **Task**: Let's find a distance between `alice` and the training point.

$$ \text{D} = \sqrt{(x_0-x_1)^2 + (y_0-y_1)^2} $$

#### **Task**: Calculate the Euclidean distance between `alice` and every point in the training set.

#### **Task:** Augment the training data table with a column containing all the distances.

#### **Task**: Find the 5 nearest neighbors.

### How do we classify the point?

#### **Task**: Find the majority class (programatically).

#### **Task:** Write a function to complete the entire algorithm!

______

In [3]:
first_banknote = banknotes.drop('Class').row(0)
first_banknote

NameError: name 'banknotes' is not defined